In [10]:
import string
import re
import pandas as pd

# Text preprocessing

Some text about why preprocessing is important.

In [11]:
df_news = pd.read_json('data/cc_gw_news_blogs_2021-10-01_2021-10-31.json')

In [1]:
## Manage upper/lower case

In [2]:
## Manage punctuation

In [5]:
## A note on encoding

In [6]:
## Two exercises

# Text searching

A note on searching strategies.

In [4]:
## Text in text method

In [7]:
## The power of regex

In [8]:
## Two exercises